In [4]:
import torch
from model import *
from torchsummary import summary    
import pickle

In [3]:
model = OneEE()
state = torch.load('trained_model.pth')
model.load_state_dict(state)

/tmp/ipykernel_24886/4280291455.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load('trained_model.pth')


<All keys matched successfully>

In [5]:
summary(model, (1, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 200, 200]             320
       BatchNorm2d-2         [-1, 32, 200, 200]              64
            Conv2d-3         [-1, 64, 200, 200]          18,496
       BatchNorm2d-4         [-1, 64, 200, 200]             128
         MaxPool2d-5         [-1, 64, 100, 100]               0
            Conv2d-6        [-1, 128, 100, 100]          73,856
       BatchNorm2d-7        [-1, 128, 100, 100]             256
            Conv2d-8        [-1, 256, 100, 100]         295,168
       BatchNorm2d-9        [-1, 256, 100, 100]             512
        MaxPool2d-10          [-1, 256, 50, 50]               0
           Conv2d-11          [-1, 128, 50, 50]         295,040
      BatchNorm2d-12          [-1, 128, 50, 50]             256
           Linear-13                    [-1, 4]       1,280,004
           Conv2d-14          [-1, 512,

The layer that has to be desactivated if the early exit is taken is Linear-19, with 1280004 paraneters, so 1280004 MACs, or 2560008 operations (multiply and sum).

In [6]:
# load the data of pickle file
import pickle

with open('metrics_EE.pkl', 'rb') as f:
    data = pickle.load(f)

In [7]:
data

{'acc_cls1': 0.7818791946308725,
 'acc_cls2': 0.8083053691275168,
 'acc_bin': 0.9499374217772215,
 'mse_reg': np.float64(0.0017734443385888697),
 'best_threshold': np.float64(0.9500000000000003),
 'best_accuracy': 0.86084142394822,
 'proportion where threshold is applied': 0.7776845637583892}

In [18]:
# over a 100 inferences

num_discount =( 1 - data['proportion where threshold is applied']) * 1280004 * 100

print(f"Over 100 inferences, aprox {num_discount:.2f} MACs will be saved")
print(f"Which represents a {num_discount / ((1280004*2 + 300000)*(1-data['proportion where threshold is applied'])):.2f} % discount in MACs over the task")

Over 100 inferences, aprox 28456464.77 MACs will be saved
Which represents a 44.76 % discount in MACs over the task
